# Training Model (DNN)

### Import 

In [18]:
import csv
import tensorflow as tf
import numpy as np

### Set Variable

In [26]:
data_x = []
data_y = []

test_x = []
test_y = []

train_x = tf.placeholder(tf.float32, shape=[None, 3])
train_y = tf.placeholder(tf.float32, shape=[None, 1])

### Read Dataset

In [27]:
def read_data():
    dataset = []
    with open('dataset/athlete_events.csv', 'r', encoding='utf-8') as f:

        reader = csv.reader(f)

        for row in reader:
            json = {}

            if row[3] == 'NA' or row[4] == 'NA' or row[5] == 'NA' or row[14] == 'NA':
                pass

            else:
                json['age'] = row[3]
                json['height'] = row[4]
                json['weight'] = row[5]
                json['medal'] = row[14]
                dataset.append(json)

        del dataset[0]

    return dataset


def set_train_data():
    
    datas = read_data()

    for row in datas:
        tmp = []

        tmp.append(float(row['age']))
        tmp.append(float(row['height']))
        tmp.append(float(row['weight']))

        data_x.append(tmp)

        if row['medal'] == 'Gold':
            data_y.append([1])
        elif row['medal'] == 'Silver':
            data_y.append([2])
        elif row['medal'] == 'Bronze':
            data_y.append([3])
        else:
            data_y.append(row['medal'])

    global test_x
    test_x = data_x[29181:]
    del data_x[29181:]
    
    global test_y
    test_y = data_y[29181:]
    del data_y[29181:]

    x_shape = np.array(data_x)
    y_shape = np.array(data_y)

    tx_shape = np.array(test_x)
    ty_shape = np.array(test_y)

    print("data_X Shape : ", x_shape.shape)
    print("data_Y Shape : ", y_shape.shape)

    print("tdata_X Shape : ", tx_shape.shape)
    print("tdata_Y Shape : ", ty_shape.shape)

### Set Dataset

In [28]:
set_train_data()

data_X Shape :  (29181, 3)
data_Y Shape :  (29181, 1)
tdata_X Shape :  (1000, 3)
tdata_Y Shape :  (1000, 1)


### Set Model

In [29]:
w1 = tf.Variable(tf.random_normal([3, 5]))
b1 = tf.Variable(tf.random_normal([5]))
l1 = tf.nn.relu(tf.add(tf.matmul(train_x, w1), b1))

w2 = tf.Variable(tf.random_normal([5, 5]))
b2 = tf.Variable(tf.random_normal([5]))
l2 = tf.nn.relu(tf.add(tf.matmul(l1, w2), b2))

w3 = tf.Variable(tf.random_normal([5, 5]))
b3 = tf.Variable(tf.random_normal([5]))
l3 = tf.nn.relu(tf.add(tf.matmul(l2, w3), b3))

w4 = tf.Variable(tf.random_normal([5, 1]))
b4 = tf.Variable(tf.random_normal([1]))
hypothesis = tf.add(tf.matmul(l3, w4), b4)

cost = tf.losses.mean_squared_error(labels=train_y, predictions=hypothesis)

learning_rate = 0.01

train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

### Set Saver

In [30]:
saver = tf.train.Saver()

save_path = "./checkpoints/olympic_nn_model.ckpt"

### Train

In [31]:
with tf.Session() as sess:
    
    # Initialize
    init = tf.global_variables_initializer()
    sess.run(init)

    # TensorBoard Setting
    cost_summ = tf.summary.scalar("cost", cost)
    summary = tf.summary.merge_all()
    writer = tf.summary.FileWriter('./log')
    writer.add_graph(sess.graph)

    # Learning Start
    print("Learning Start!")
    for step in range(10001):
        cost_v, _ = sess.run([cost, train], feed_dict={train_x: data_x, train_y: data_y})
        if step % 200 == 0:
            print("STEP :", step, "Cost :", cost_v)
            
            
    # Model Save
    print("Saving Model...")        
    saver.save(sess, save_path)
    print("Save Successful!")

    # Model Testing
    print("Model Test...")
    result = sess.run([hypothesis], feed_dict={train_x: [[22.0, 185.0, 82.0]]})
    print(result)

    # Check Accuracy
    print("Checking Accuracy...")
    correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(train_y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print("Accuracy :",sess.run(accuracy, feed_dict={train_x: test_x, train_y: test_y}) * 100,"%")

    print("Finished")

Learning Start!
STEP : 0 Cost : 36075.664
STEP : 200 Cost : 4.5425906
STEP : 400 Cost : 3.6861656
STEP : 600 Cost : 2.8166692
STEP : 800 Cost : 2.0745845
STEP : 1000 Cost : 1.5152005
STEP : 1200 Cost : 1.1366699
STEP : 1400 Cost : 0.9056429
STEP : 1600 Cost : 0.77858955
STEP : 1800 Cost : 0.7158967
STEP : 2000 Cost : 0.68832475
STEP : 2200 Cost : 0.6776087
STEP : 2400 Cost : 0.67396605
STEP : 2600 Cost : 0.67289597
STEP : 2800 Cost : 0.6726281
STEP : 3000 Cost : 0.6725718
STEP : 3200 Cost : 0.67256206
STEP : 3400 Cost : 0.67256063
STEP : 3600 Cost : 0.6725605
STEP : 3800 Cost : 0.6725605
STEP : 4000 Cost : 0.6725605
STEP : 4200 Cost : 0.6725605
STEP : 4400 Cost : 0.6725605
STEP : 4600 Cost : 0.6725606
STEP : 4800 Cost : 0.6725605
STEP : 5000 Cost : 0.6725605
STEP : 5200 Cost : 0.6725605
STEP : 5400 Cost : 0.6725605
STEP : 5600 Cost : 0.6725605
STEP : 5800 Cost : 0.6725605
STEP : 6000 Cost : 0.6725605
STEP : 6200 Cost : 0.6725605
STEP : 6400 Cost : 0.6725605
STEP : 6600 Cost : 0.6725604

### Using Model

In [38]:
sess = tf.Session()

# Model Restore
saver.restore(sess, save_path)
    
# Input Values
weight = input("Weight : ")
height = input("Height : ")
old = input("How Old : ")

# Model Testing
print("Model Test...")
result = sess.run(hypothesis, feed_dict={train_x: [[old, height, weight]]})[0][0]

print(result)

if round(result) == 1:
    print("Gold Medal")

elif round(result) == 2:
    print("Silver Medal")
    
elif round(result) == 3:
    print("Bronze Medal")

else:
    print("No Medal")

print("Finished")
    

INFO:tensorflow:Restoring parameters from ./checkpoints/olympic_nn_model.ckpt
Weight : 75
Height : 182
How Old : 31
Model Test...
1.9993827
Silver Medal
Finished
